<a href="https://colab.research.google.com/github/OscarBedford/MLCourse_Weekly_Exercises/blob/main/Exercise_13_5_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 5- Same as 3, but creating 2 higher-level categories based on median-split of the
‘delay’ variable supplied by the OASIS dataset. 500 MCMC iterations is enough. Report
output of PyMC3.summary(credible_interval=0.90).

In [ ]:
%%capture
!pip install nilearn
!pip install pymc3

In [ ]:
%%capture
import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
import nibabel as nib

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pymc3 as pm
import scipy.stats as stats
import arviz as az
from pymc3 import Bernoulli, Model, HalfCauchy
from pymc3 import invlogit, sample, summary
from numpy import mean
from numpy import std
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_decomposition import PLSRegression as PLSR
from sklearn.cross_decomposition import CCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression as LR

In [ ]:
%%capture
brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
yeo = datasets.fetch_atlas_schaefer_2018(n_rois=100) # this needs to be modified for some tasks
masker = NiftiLabelsMasker(labels_img=yeo.maps, standardize=True, memory='nilearn_cache')
input_variables = masker.fit_transform(brain_data.gray_matter_maps)
output_variable = np.array(brain_data.ext_vars.mf == 'F', dtype=int) # gives 1 for females and 0 for males

In [ ]:
# We will extract delays from brain_data object
delays = brain_data.ext_vars.delay
print(delays)

[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan]


NOTE: the delay column is a collection of NaNs. Therefore, we will set up the model as indicated, but we cannot really trust the following results.

In [ ]:
# We median-split the sample to find the value that divides people into low and high delays
delay_median = np.median(delays)

In [ ]:
# Now we can proceed to defining the delay hyperprior, and then age as in Exercise 4
with pm.Model() as model:

    # We define the binary hyperprior based on delay and call it "delay"
    delay = pm.Bernoulli('delay', p=0.5, shape=len(delays))
    low = delay[delays < delay_median]
    high = delay[delays >= delay_median]

    # We now extract ages from the brain_data object
    ages = brain_data.ext_vars.age
    # We median-split the sample using numpy in order to find the value that divides people into low and high age
    age_median = np.median(ages)

    # We define a binary hyperprior based on age and we call it "age"
    age = pm.Bernoulli('age', p=0.5, shape=len(ages))
    low = age[ages < age_median]
    high = age[ages >= age_median]

    # We define the HalfCauchy hyperprior for the variance component of the lower-level region slopes (betas)
    hyperprior = pm.HalfCauchy('hyperprior', beta=1, shape=100)

    # We define intercept term as normal distribution with prior (mu=0, sd=1)
    intercept = pm.Normal('intercept', mu=0, sd=1)

    # We define the bottom-level region slopes (betas) as a normal distribution with priors mu=0 and sd=hyperprior (HalfCauchy)
    beta_coefficients = pm.Normal('beta_coefficients', mu=0, sd=hyperprior, observed=input_variables)

    # We calculate the prior probabilities using the invlogit function as specificed in the instructions
    probabilities = pm.invlogit(intercept + input_variables @ beta_coefficients + delay + age)

    # We define likelihood as a another normal distribution with mean equal to probabilities and sd=1
    likelihood = pm.Normal('likelihood', mu=probabilities, sd=1, observed=output_variable)

    # We sample from the posterior distribution using 500 MCMC draws
    trace = pm.sample(500, return_inferencedata=True)

    # We're ready to summarize the results of the MCMC sampling
    stats = pm.summary(trace, hdi_prob=0.9)

In [ ]:
# Let's take a look at the summary table
from google.colab import data_table
data_table.DataTable(stats, include_index=True, num_rows_per_page=301)

,mean,sd,hdi_5%,hdi_95%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
delay[0],0.661,0.474,0.000,1.000,0.012,0.009,1481.0,1000.0,1.00
delay[1],0.863,0.344,0.000,1.000,0.012,0.009,782.0,782.0,1.00
delay[2],0.784,0.412,0.000,1.000,0.014,0.010,897.0,897.0,1.00
delay[3],0.073,0.260,0.000,0.000,0.010,0.007,703.0,703.0,1.00
delay[4],0.125,0.331,0.000,1.000,0.011,0.008,922.0,922.0,1.00
...,...,...,...,...,...,...,...,...,...
hyperprior[95],1.009,0.070,0.896,1.118,0.002,0.001,2189.0,707.0,1.01
hyperprior[96],1.006,0.067,0.897,1.112,0.002,0.001,1847.0,748.0,1.01
hyperprior[97],1.008,0.073,0.898,1.131,0.002,0.001,2326.0,919.0,1.00
hyperprior[98],1.007,0.067,0.899,1.119,0.002,0.001,2216.0,488.0,1.00
